In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://github.com/PacktWorkshops/The-Deep-Learning-Workshop/raw/master/Chapter03/Datasets/Exercise3.03/cats_and_dogs_filtered.zip'

In [3]:
zip_dir = tf.keras.utils.get_file('cats_and_dogs.zip', origin=file_url, extract=True)

68616192/68606236 [==============================] - 5s 0us/step


In [4]:
zip_dir

'/Users/LNonyane/.keras/datasets/cats_and_dogs.zip'

In [5]:
import pathlib

In [6]:
path = pathlib.Path(zip_dir).parent / 'cats_and_dogs_filtered'

In [7]:
print(path.parent)

/Users/LNonyane/.keras/datasets


In [8]:
[x for x in path.iterdir() if x.is_dir()]

[PosixPath('/Users/LNonyane/.keras/datasets/cats_and_dogs_filtered/train'),
 PosixPath('/Users/LNonyane/.keras/datasets/cats_and_dogs_filtered/validation')]

In [9]:
train_dir = path / 'train'
validation_dir = path / 'validation'

In [10]:
train_cats_dir = train_dir / 'cats'
train_dogs_dir = train_dir / 'dogs'
validation_cats_dir = validation_dir / 'cats'
validation_dogs_dir = validation_dir / 'dogs'

In [11]:
import os

In [12]:
total_train = len(os.listdir(train_cats_dir)) + len(os.listdir(train_dogs_dir))
total_val = len(os.listdir(validation_cats_dir)) + len(os.listdir(validation_dogs_dir))

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [15]:
batch_size = 16
img_height = 100
img_width = 100

In [17]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size, directory=train_dir, 
                                                          shuffle=True, target_size=(img_height, img_width), 
                                                          class_mode='binary')

Found 2000 images belonging to 2 classes.


In [18]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size, directory=validation_dir, 
                                                             shuffle=True, target_size=(img_height, img_width), 
                                                             class_mode='binary')

Found 1000 images belonging to 2 classes.


In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [20]:
np.random.seed(8)
tf.random.set_seed(8)

In [22]:
model = tf.keras.Sequential([layers.Conv2D(64, (3), activation='relu', input_shape=(img_height, img_width, 3)), # A convolution layer with 64 kernels of shape 3, ReLU as the activation function, and the required input dimensions
                            layers.MaxPool2D(), # a max pooling layer
                            layers.Conv2D(128, (3), activation='relu'), # a convolution layer with 128 kernels of shape 3 and ReLU as the activation function
                            layers.MaxPool2D(), # a max pooling layer
                            layers.Flatten(), # a flatten layer
                            layers.Dense(128, activation='relu'), # a fully connected layer with 128 units and ReLU as the activation function
                            layers.Dense(1, activation='sigmoid')]) # a fully connected layer with 1 unit and sigmoid as the activation function

In [23]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [27]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 67712)             0         
                                                                 
 dense (Dense)               (None, 128)              

In [28]:
model.fit(train_data_gen, steps_per_epoch=total_train // batch_size, epochs=5, validation_data=val_data_gen, 
                   validation_steps=total_val // batch_size)

Epoch 1/5
125/125 [==============================] - 24s 190ms/step - loss: 0.7337 - accuracy: 0.5410 - val_loss: 0.6845 - val_accuracy: 0.5222
Epoch 2/5
125/125 [==============================] - 23s 184ms/step - loss: 0.6530 - accuracy: 0.6360 - val_loss: 0.6351 - val_accuracy: 0.6512
Epoch 3/5
125/125 [==============================] - 23s 184ms/step - loss: 0.5713 - accuracy: 0.6990 - val_loss: 0.6668 - val_accuracy: 0.6240
Epoch 4/5
125/125 [==============================] - 23s 184ms/step - loss: 0.4585 - accuracy: 0.7825 - val_loss: 0.6537 - val_accuracy: 0.7006
Epoch 5/5
125/125 [==============================] - 23s 182ms/step - loss: 0.3174 - accuracy: 0.8575 - val_loss: 0.7862 - val_accuracy: 0.6623


In [29]:
# reduce overfitting adding another full layer with 64 units
model2 = tf.keras.Sequential([layers.Conv2D(64, (3), activation='relu', input_shape=(img_height, img_width, 3)), # A convolution layer with 64 kernels of shape 3, ReLU as the activation function, and the required input dimensions
                            layers.MaxPool2D(), # a max pooling layer
                            layers.Conv2D(128, (3), activation='relu'), # a convolution layer with 128 kernels of shape 3 and ReLU as the activation function
                            layers.MaxPool2D(), # a max pooling layer
                            layers.Flatten(), # a flatten layer
                            layers.Dense(128, activation='relu'), # a fully connected layer with 128 units and ReLU as the activation function
                            layers.Dense(64, activation='relu'), # a fully connected layer with 64 units and ReLU as the activation function
                            layers.Dense(1, activation='sigmoid')]) # a fully connected layer with 1 unit and sigmoid as the activation function

In [32]:
model2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [33]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 23, 23, 128)      0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 67712)             0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [34]:
model2.fit(train_data_gen, steps_per_epoch=total_train // batch_size, epochs=5, validation_data=val_data_gen, 
                   validation_steps=total_val // batch_size)

Epoch 1/5
125/125 [==============================] - 24s 185ms/step - loss: 0.7609 - accuracy: 0.4770 - val_loss: 0.6931 - val_accuracy: 0.5010
Epoch 2/5
125/125 [==============================] - 23s 183ms/step - loss: 0.6934 - accuracy: 0.4830 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/5
125/125 [==============================] - 23s 185ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/5
125/125 [==============================] - 24s 194ms/step - loss: 0.6935 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/5
125/125 [==============================] - 28s 222ms/step - loss: 0.6930 - accuracy: 0.5005 - val_loss: 0.6932 - val_accuracy: 0.5000
